# Output part of infinite matter dataframe as LaTeX table

This notebook generates Tables II and III in the Appendix of _Quantifying uncertainties and correlations in the nuclear-matter equation of state_ by [BUQEYE](https://buqeye.github.io/) members Christian Drischler, Jordan Melendez, Dick Furnstahl, and Daniel Phillips (see [[arXiv:2004.07805]](https://arxiv.org/abs/2004.07805)). 

Data is read in from nuclear matter calculations (both SNM and PNM) and outputs total energies at each EFT order as a function of both density and Fermi momentum in the form of LaTeX tables. It uses pandas to manipulate the data and dump it to LaTeX. The details are easily modified.


In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np


## Data import and processing

The calculations for infinite matter are stored in a standardized csv file in the data directory.  Both symmetric nuclear matter (SNM) and pure neutron matter (PNN) are included.  The fields in the file are

  | field | units | description |
  | :---: | :---: | :---- |  
  |  kf   | $$\text{fm}^{-1}$$ | Fermi momentum. |
  |  n    | $$\text{fm}^{-3}$$ | Density. |
  | Kin   |   MeV  | Kinetic energy. |
  | MBPT_HF | MeV  | Hartree-Fock energy (leading order in MBPT). |
  | MBPT_2 |  MeV  | 2nd-order contribution in MBPT (not total). |
  | MBPT_3 |  MeV  | 3rd-order contribution in MBPT (not total). |
  | MBPT_4 |  MeV  | 4th-order contribution in MBPT (not total). |
  | total  |  MeV  | Total energy (sum of all contributions). |
  | Lambda |  MeV  | Regulator parameter.
  | OrderEFT |     | Order of the EFT: LO, NLO, N2LO, N3LO |
  | Body   |       | Two-body only (NN) or two-plus-three (NN+3N)
  |  x     |       | Proton fraction: 0.5 is SNM; 0.0 is PNM. 
  | fit    |       | Index for the fit.



The following is commented code from another notebook that identifies the indexes for fits from arXiv:1710.08220.

In [2]:
# EFT orders LO, NLO, N2LO, N3LO
#orders = np.array([0, 2, 3, 4])  # powers of Q
# body = 'NN-only'
# body = 'NN+3N'
# Lambda = 450

# Specify by index what fits will be used for the 3NF [N2LO, N3LO]
#  The indices follow the fits in Fig. 3 of arXiv:1710.08220
# fits = {450: [1, 7], 500: [4, 10]}
# fits_B = {450: [2, 8], 500: [5, 11]}
# fits_C = {450: [3, 9], 500: [6, 12]}



In [3]:
# Replace the following with the path to the desired data file
data_file = '../data/all_matter_data.csv'

# Read infinite matter data from specified csv file into a dataframe df
df = pd.read_csv(data_file)

# Make copies of the dataframe for experiments
df_kin = df.copy()
df_all = df.copy()

# Convert differences to total energy prediction at each MBPT order
#mbpt_orders = ['Kin', 'MBPT_HF', 'MBPT_2', 'MBPT_3', 'MBPT_4']
#df[mbpt_orders] = df[mbpt_orders].apply(np.cumsum, axis=1)

### Replacements in column names or values

These are minor fixes to the files that we fix by hand.

In [4]:
df = df.replace({'OrderEFT' : 'NLO'}, 'N1LO') # makes columns align correctly
# We notice some truncation problems we fix by hand.
df = df.replace({'kf' : 0.904594}, 0.904590)  # fix a truncation difference
df = df.replace({'kf' : 0.961274}, 0.961270)  # fix a truncation difference

In [5]:
# For our basic tables we only need the 'total' column, so delete the other energies.
pop_list = ['Kin', 'MBPT_HF', 'MBPT_2', 'MBPT_3', 'MBPT_4']
for col in pop_list:
    df.pop(col)

# check it
df

,kf,n,total,Lambda,OrderEFT,Body,x,fit
0,1.13972,0.05,8.643290,450,LO,NN-only,0.0,NaN
1,1.21113,0.06,9.718110,450,LO,NN-only,0.0,NaN
2,1.27499,0.07,10.732550,450,LO,NN-only,0.0,NaN
3,1.33302,0.08,11.689420,450,LO,NN-only,0.0,NaN
4,1.38640,0.09,12.603940,450,LO,NN-only,0.0,NaN
...,...,...,...,...,...,...,...,...
811,1.36023,0.17,-14.730505,500,N3LO,NN+3N,0.5,12.0
812,1.38640,0.18,-14.702775,500,N3LO,NN+3N,0.5,12.0
813,1.41161,0.19,-14.575082,500,N3LO,NN+3N,0.5,12.0
814,1.43595,0.20,-14.335640,500,N3LO,NN+3N,0.5,12.0


In [6]:
def dump_to_file(df, output_file, kf_column='snm'):
    """
    Output adjusted dataframe to a file in LaTeX format for tables.
    Modify the format here or generalize for different looks.
    """
    with open(output_file, 'w') as of:
        of.write('% description\n')
        of.write(df.to_latex(
                 index=False,
                 formatters={'LO':'${:,.2f}$'.format,
                             'N1LO':'${:,.2f}$'.format,
                             'N2LO':'${:,.2f}$'.format,
                             'N3LO':'${:,.2f}$'.format,
                             'kf_snm':'${:,.2f}$'.format},
                 columns=['n', kf_column, 'LO', 'N1LO', 'N2LO', 'N3LO'],
                 escape=False
                ))
    

In [7]:
Lambdas = (450, 500)

for Lambda in Lambdas:
    s_Lambda = (df['Lambda']==Lambda) 
    # s_x = (df['x']==0.5) | (df['x']==0.0)
    s_x_SNM = df['x']==0.5  # select SNM (proton fraction 1/2)
    s_x_PNM = df['x']==0.0  # select PNM (proton fraction 0) 
    s_Body = df['Body']=='NN+3N'  # 'NN+3N' or 'NN-only'
    s_n = True #  df['n']==0.5    # could select a subset of densities
    
    # For the 'fit', the LO and NLO values are NaN, so use pd.isna
    if Lambda == 450:
        s_fit = df['fit'].isin([1.0, 7.0]) | pd.isna(df['fit'])
    elif Lambda == 500:
        s_fit = df['fit'].isin([4.0, 10.0]) | pd.isna(df['fit']) 
    
    # Make a table just for SNM and a specified Lamba
    df_SNM = df.loc[s_Lambda & s_x_SNM & s_Body & s_n & s_fit ]
    df_SNM.pop('x')   # we don't want 'x' anymore
    df_SNM = df_SNM.rename(columns={"kf": "kf_snm"})
    
    # Make a table just for PNM and a specified Lamba
    df_PNM = df.loc[s_Lambda & s_x_PNM & s_Body & s_n & s_fit ]
    df_PNM.pop('x')   # we don't want 'x' anymore
    df_PNM = df_PNM.rename(columns={"kf": "kf_pnm"})
    
    # Check the tables
    df_SNM
    
    # Pivoting here means to take the row entries for OrderEFT and make them columns
    df_SNM_pivoted = df_SNM.pivot_table(values='total', columns='OrderEFT', 
                                        index=('n','kf_snm')).reset_index()
    df_PNM_pivoted = df_PNM.pivot_table(values='total', columns='OrderEFT', 
                                        index=('n','kf_pnm')).reset_index()
    
    SNM_output_file = f'SNM_table_Lambda{Lambda}.tex' 
    dump_to_file(df_SNM_pivoted, SNM_output_file, kf_column='kf_snm')

    PNM_output_file = f'PNM_table_Lambda{Lambda}.tex'
    dump_to_file(df_PNM_pivoted, PNM_output_file, kf_column='kf_pnm')
